## ***beyond the DAG*** - patterns in more complicated workflows

### Concepts
- **Flow of Flows**: parent orchestrator patterns and subflows

- **Breaking the DAG**: changing the control flow on the fly

## Extending the `nike_flow` from earlier
Now imagine we have many URLs pointing to Nike shoes that we like, and we want to save all the shoes in our budget. 

We can create an instance of our `nike_flow` for each `url`, all wrapped in a new function we call `parent_flow`:


```python
@flow
def parent_flow(urls: List[str], budget: int) -> None:
    for url in urls:
        nike_flow(url, budget)
```


In Prefect language, this is what we call a [subflow](https://orion-docs.prefect.io/concepts/flows/#subflows) pattern. 

Using such a pattern, we can dynamically change the execution of the `parent_flow` based on the results we get from each `nike_flow` instance (subflow) ***during runtime***. 

This what we mean by *"breaking the DAG"* since representing workflows as DAGs requires knowing the execution plan ***before runtime***.



## our flow of flows

Imports!

In [2]:
from bs4 import BeautifulSoup
from prefect import flow, task
from typing import List
import requests
import re

Let's use our old friend `find_nice_price` from earlier:

In [3]:
@task(retries=3, retry_delay_seconds=10)
def find_nike_price(url: str) -> int:
    k = requests.get(url).text
    soup = BeautifulSoup(k,'html.parser')
    price_string = soup.find('div', {"class":"product-price"}).text.replace(' ','')
    price = re.search('[0-9]+', price_string).group(0)
    return int(price)

Let's define a `task` called `inBudget`, similar to `compare_price` from before, only it will `return` a `bool` that will inform our downstream tasks:

In [4]:
@task
def inBudget(price: int, budget: int) -> bool:
    return price <= budget

Now with `inBudget` defined, we use it decide whether we want to `save` that shoe's `url` or not:

In [5]:
@task
def save(url: str, to: str = 'shoes_in_my_budget.md') -> None:
    with open(to, 'w') as f:
        link = f"[{url.split('/t/')[-1]}]({url})\n\n"
        f.write(link)

... where we write the `url` to a markdown file `if` the corresponding shoe is `inBudget`. Now, our `nike_flow` looks like:

In [6]:
@flow
def nike_flow(url: str, budget: int) -> None:
    price = find_nike_price(url)
    
    if inBudget(price, budget).result():
        save(url)

Because we can arbitrarily nest `flow` calls in the newest iteration of Prefect, kicking off subflows is as easy as using a `for` loop:

In [7]:
@flow
def parent_flow(urls: List[str], budget: int) -> None:
    for url in urls:
        nike_flow(url, budget)

So we can now kick off our whole workflow by running the following cell:

In [ ]:
urls = [
    "https://www.nike.com/t/air-max-270-womens-shoes-Pgb94t/AH6789-601",
    "https://www.nike.com/t/air-max-terrascape-90-mens-shoes-R6r8hB/DH2973-100",
    "https://www.nike.com/t/pegasus-trail-3-gore-tex-mens-running-shoes-HG005k/DR0137-200"
]

budget = 150

parent_flow(urls, budget)

## What did it do?
Let's look at the output found in the [shoes_in_my_budget](shoes_in_my_budget.md) markdown file...


We see the links to the URLs pointing to shoes that have a `price` that is less than our `budget`.

### Benefits of using subflow patterns when applicable

- Ability to orchestrate many instances of a complex, stateful workflow without duplicating work

- Maintain visibility into each of the subflow processes via the UI

<hr>

### Discussion: What else might we want to use a subflow pattern for?

#### Q&A
